In [ ]:
from __future__ import division

import argparse
import scipy.io as sio
import numpy as np

import os
import pandas
import re
import sys
import time

from pandas import Series, DataFrame
from sklearn.cross_validation import StratifiedKFold

from tengwar.data import extract_features_with_sliding_window, get_resampling_features

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('input', type=unicode)
parser.add_argument('output', type=unicode)
parser.add_argument('--resample_rate', '-r', type=int, default=1)
parser.add_argument('--min_length', '-m', type=float, default=12)
parser.add_argument('--target_length', '-l', type=float, default=48)

import __main__
if not hasattr(__main__, '__file__'):
    import platform
    users = '/Users/' if platform.system()=='Darwin' else '/home/'
    args = parser.parse_args(args=[ users + 'mldata/medical/physionet-challenge2012/physionet_challenge2012_merged_bp.npz',
                                    users + 'mldata/KDD/physionet'
                                  ])
    %pylab inline
else:
    args = parser.parse_args()

In [ ]:
sys.stdout.write('Loading episode lengths...')
sys.stdout.flush()
ndata = np.load(args.input)
T = ndata['tsraw']
ylos = ndata['ylos'].astype(float)
ylos[ylos==-1] = np.nan

sys.stdout.write('keeping only >={0} hours...'.format(args.min_length))
sys.stdout.flush()
lengths = np.nanmin(np.vstack((np.array([ t.max()/60 for t in T ]), ylos*24)),axis=0)
keepix = (lengths>=args.min_length)

sys.stdout.write('{0} episodes ({1:.2f}%)\n'.format(keepix.sum(), keepix.mean()*100))

In [ ]:
sys.stdout.write('Loading data...')
sys.stdout.flush()
el = time.clock()

T = T[keepix]
ylos = ylos[keepix]

X = ndata['Xraw'][keepix]
Y = np.vstack(( ndata['ym'][keepix].astype(float),
                ylos,
                ndata['ysurv'][keepix].astype(float) )).T

recordid = ndata['recordid'][keepix]
Ep = np.arange(X.shape[0])

icutype = ndata['icutype'][keepix]
icutype_bin = np.zeros((icutype.shape[0],4))
for i in np.arange(icutype.shape[0]):
    icutype_bin[i,icutype[i]-1] = 1

Xstatic = np.vstack(( ndata['age'][keepix],
                      ndata['gender'][keepix],
                      ndata['height'][keepix],
                      icutype_bin.T,
                      np.array([ np.any(x[:,18][~np.isnan(x[:,18])]>0) for x in X ]),
                      ndata['weight'][keepix] )).T
Xstatic[Xstatic == -1] = np.nan
Xstatic_names = [ 'age', 'gender', 'height' ]
Xstatic_names.extend([ 'icutype{0}'.format(u+1) for u in np.arange(4) ])
Xstatic_names.extend([ 'icutype', 'mechvent', 'weight' ])

scores = np.vstack(( ndata['saps1'][keepix], ndata['sofa'][keepix] )).T
scores_names = [ 'saps1', 'sofa' ]

el = time.clock() - el
sys.stdout.write('DONE, took {0:2f}s\n'.format(el))

In [ ]:
msmts_per_ep = np.vstack([ np.sum(~np.isnan(x),axis=0) for x in X ])
hours_per_ep = np.array([ t.max()/60 for t in T ])

msmts_per_hour_overall = msmts_per_ep.sum(axis=0) / hours_per_ep.sum()
#print 'MSMTS/HR, OVERALL', msmts_per_hour_overall
TEMP = msmts_per_ep.astype(float)
TEMP[TEMP==0] = np.nan
msmts_per_hour_average = np.nanmean(TEMP / hours_per_ep[:,None],axis=0)
print 'MSMTS/HR, AVERAGE', msmts_per_hour_average
print ''

ix = np.argsort(msmts_per_hour_average)
_ = plt.figure()
_ = plt.bar(np.arange(msmts_per_hour_average.shape[0]), msmts_per_hour_average[ix], width=0.9, color='r')
_ = plt.xticks([])
#_ = plt.xticks(np.arange(hours_per_msmt_average.shape[0]), np.arange(hours_per_msmt_average.shape[0])[ix]+1)
_ = plt.title('Avg msmts/hour (ignoring missing TS)')
plt.savefig('sampling-msmts_hours.png')

hours_per_msmt_overall = hours_per_ep.sum() / msmts_per_ep.sum(axis=0)
#print 'HRS/MSMT, OVERALL', hours_per_msmt_overall
hours_per_msmt_average = hours_per_ep[:,None] / msmts_per_ep
hours_per_msmt_average[np.isinf(hours_per_msmt_average)] = np.nan
hours_per_msmt_average = np.nanmean(hours_per_msmt_average,axis=0)
print 'HRS/MSMT, AVERAGE', hours_per_msmt_average

missingness = (msmts_per_ep==0).mean(axis=0)
ix = np.argsort(missingness)
_ = plt.figure()
_ = plt.bar(np.arange(missingness.shape[0]), missingness[ix], width=0.9, color='g')
_ = plt.xticks([])
_ = plt.title('Fraction of episodes with 0 msmts')
plt.savefig('missingness.png')

ix = np.argsort(hours_per_msmt_average)
_ = plt.figure()
_ = plt.bar(np.arange(hours_per_msmt_average.shape[0]), hours_per_msmt_average[ix], width=0.9, color='b')
#_ = plt.xticks(np.arange(hours_per_msmt_average.shape[0]), np.arange(hours_per_msmt_average.shape[0])[ix]+1)
_ = plt.title('Avg hours/msmt (ignoring missing TS)')
_ = plt.xticks([])
plt.savefig('sampling-hours_msmts.png')

df = DataFrame(data={'VariableId': np.arange(msmts_per_hour_overall.shape[0])+1, 'Missing': missingness,
                     'MsmtPerHourOverall': msmts_per_hour_overall, 'MsmtPerHourAverage': msmts_per_hour_average,
                     'HoursBtwnMsmtOverall': hours_per_msmt_overall, 'HoursBtwnMsmtAverage': hours_per_msmt_average})
df.sort(columns=['Missing','HoursBtwnMsmtAverage','MsmtPerHourAverage'], ascending=[True,True,False], inplace=True)
df.set_index('VariableId', inplace=True)
df = df[['Missing','HoursBtwnMsmtAverage','HoursBtwnMsmtOverall','MsmtPerHourAverage','MsmtPerHourOverall']]
df.to_excel('sampling-rates-etc.xls')
df.to_csv('sampling-rates-etc.csv')

In [ ]:
sys.stdout.write('Determining variable stats/ranges...')
sys.stdout.flush()
el = time.clock()

stats = np.zeros((5,X[0].shape[1],3))
for u in np.unique(icutype):
    Xmsmts = np.vstack(X[icutype==u])
    stats[u,:] = np.nanpercentile(Xmsmts, [0.01, 50, 99.99], axis=0).T
    
Xmsmts = np.vstack(X)
stats[0,:] = np.nanpercentile(Xmsmts, [0.01, 50, 99.99], axis=0).T

ranges = np.hstack((np.arange(stats.shape[1])[:,None]+1, stats[0,:,:]))

el = time.clock() - el
sys.stdout.write('DONE, took {0:2f}s\n'.format(el))

In [ ]:
print 'Empirical Min, Normal, Max'
print DataFrame(ranges).set_index(0)
print ''
print 'Correcting <0 Low values and variable 19 (0)'
ranges[ranges[:,1]<0,1] = 0
ranges[ranges[:,0]==19,1] = 0

print ''
print 'Correcting Normal values for variables 10 (0.21), 11 (13), 19 (0)'
ranges[ranges[:,0]==10,2] = 0.21
ranges[ranges[:,0]==11,2] = 13
ranges[ranges[:,0]==19,2] = 0
ranges[ranges[:,0]==36,2] = 0.5
ranges[ranges[:,0]==38,1:3] = 0
ranges[ranges[:,0]==39,2] = 0

print ''
print 'Final Ranges'
print DataFrame(ranges).set_index(0)

In [ ]:
xix = (ranges[:,3]-ranges[:,1]) != 0
print 'Removing {0} variables with no range, weight (var 33) b/c of noise'.format((~xix).sum())
xix[32] = False
ranges = ranges[xix,:]
stats  = stats[:,xix,:]

In [ ]:
sys.stdout.write('Determining static variable stats/ranges...')
sys.stdout.flush()
el = time.clock()

stats_static = np.zeros((5,Xstatic.shape[1],3))
for u in np.unique(icutype):
    stats_static[u,:] = np.nanpercentile(Xstatic[icutype==u,:], [0.01, 50, 99.99], axis=0).T
    
stats_static[0,:] = np.nanpercentile(Xstatic, [0.01, 50, 99.99], axis=0).T

ranges_static = np.hstack((np.arange(stats_static.shape[1])[:,None]+1, stats_static[0,:,:]))

el = time.clock() - el
sys.stdout.write('DONE, took {0:2f}s\n'.format(el))

In [ ]:
ranges_static[ranges_static[:,0]==2,2] = 0.5
ranges_static[ranges_static[:,0]==4,2] = 0
ranges_static[ranges_static[:,0]==5,2] = 0

In [ ]:
X_list = np.where(xix)[0] + 1

In [ ]:
Xmsmts = np.vstack(X)
for vi,x in enumerate(Xmsmts[:,xix].T):
    fg = plt.figure()
    _ = plt.hist(x[~np.isnan(x)], bins=100, cumulative=False, range=(ranges[vi,1], ranges[vi,3]))
    plt.title('Variable {0}, before rescale ({1} > 0.75)'.format(X_list[vi], np.nansum(x>=0.75*ranges[vi,3])))
    plt.savefig(os.path.join('var{0:02d}-values.png'.format(X_list[vi])))

In [ ]:
sys.stdout.write('Rescaling all variables to empirical range, removing outliers...')
sys.stdout.flush()
Xraw = X.copy()
X = np.array([(x[:,xix]-ranges[:,1]) / (ranges[:,3] - ranges[:,1]) for x in X ])
for x in X:
    x[x<0] = 0
    x[x>1] = 1

sys.stdout.write('DONE!\n')

In [ ]:
Xmsmts = np.vstack(X)
for vi,x in enumerate(Xmsmts.T):
    fg = plt.figure()
    _ = plt.hist(x[~np.isnan(x)], bins=100, cumulative=False, range=(0,1))
    plt.title('Variable {0}, after rescale ({1} > 0.75)'.format(X_list[vi], np.nansum(x>=0.75)))
    plt.savefig(os.path.join('var{0:02d}-values-rescaled.png'.format(X_list[vi])))

In [ ]:
sys.stdout.write('Rescaling all static variables to empirical range, removing outliers...')
sys.stdout.flush()
Xstatic_raw = Xstatic.copy()
Xstatic = (Xstatic-ranges_static[:,1]) / (ranges_static[:,3] - ranges_static[:,1])
Xstatic[Xstatic<0] = 0
Xstatic[Xstatic>1] = 1

sys.stdout.write('DONE!\n')

In [ ]:
Xstatic_miss = Xstatic.copy()
for vi in np.arange(Xstatic_miss.shape[1]):
    ix = np.isnan(Xstatic[:,vi])
    Xstatic[ix,vi] = ranges_static[vi,2]

In [ ]:
sys.stdout.write('Creating resampled, missing time series...')
sys.stdout.flush()
el = time.clock()

Xmiss = [ extract_features_with_sliding_window(x, fn=get_resampling_features, timestamps=t/60,
                                               max_length=args.target_length, impute=False)
            for (x,t) in zip(X,T) ]
Xmiss = np.array([ np.squeeze(np.rollaxis(x, axis=2)) for x in Xmiss ])

el = time.clock() - el
sys.stdout.write('DONE, took {0:2f}s\n'.format(el))

In [ ]:
sys.stdout.write('Creating resampled, imputed time series...')
sys.stdout.flush()
el = time.clock()

Ximpute = [ extract_features_with_sliding_window(x, fn=get_resampling_features, timestamps=t/60,
                                                 max_length=args.target_length, impute=True)
            for (x,t) in zip(X,T) ]
Ximpute = np.array([ np.squeeze(np.rollaxis(x, axis=2)) for x in Ximpute ])

el = time.clock() - el
sys.stdout.write('DONE, took {0:2f}s\n'.format(el))

In [ ]:
Timpute = np.array([ x.shape[0] if len(x.shape) == 2 else 1 for x in Ximpute ])

In [ ]:
sys.stdout.write('Imputing missing time series with unit specific normals...')
sys.stdout.flush()
el = time.clock()

XimputeUnit = Ximpute.copy()
for xi in np.arange(XimputeUnit.shape[0]):
    u = icutype[xi]
    x = XimputeUnit[xi]
    for vi in np.arange(x.shape[1]):
        if vi == 9:
            XimputeUnit[xi][np.isnan(x[:,vi]),vi] = (ranges[9,2] - ranges[9,1]) / (ranges[9,3] - ranges[9,1])
        elif vi == 10:
            XimputeUnit[xi][np.isnan(x[:,vi]),vi] = (ranges[10,2] - ranges[10,1]) / (ranges[10,3] - ranges[10,1])
        elif vi == 18:
            XimputeUnit[xi][np.isnan(x[:,vi]),vi] = (ranges[18,2] - ranges[18,1]) / (ranges[18,3] - ranges[18,1])
        else:
            XimputeUnit[xi][np.isnan(x[:,vi]),vi] = (stats[u,vi,1] - ranges[vi,1]) / (ranges[vi,3] - ranges[vi,1])

el = time.clock() - el
sys.stdout.write('DONE, took {0:2f}s\n'.format(el))

In [ ]:
sys.stdout.write('Imputing missing time series with unit specific normals...')
sys.stdout.flush()
el = time.clock()

for xi in np.arange(Ximpute.shape[0]):
    x = Ximpute[xi]
    for vi in np.arange(x.shape[1]):
        Ximpute[xi][np.isnan(x[:,vi]),vi] = (ranges[vi,2] - ranges[vi,1]) / (ranges[vi,3] - ranges[vi,1])
el = time.clock() - el
sys.stdout.write('DONE, took {0:2f}s\n'.format(el))

In [ ]:
ix = ~np.isnan(ymort) & ~np.isnan(ylos) & ~np.isnan(icutype)

temp = ysurv.copy()
temp[temp==-999] = 999
temp = temp-ylos
Th = range(1,11)
cc = np.array([np.corrcoef(np.vstack((ymort[ix],(temp[ix]<th).astype(float))))[0,1] for th in Th])
th = Th[np.where(cc<1.0)[0][0]]
ysurvbin = (temp<th).astype(float)
lsurvbin = 'survival<{0}'.format(th)

#Th = range(3,11)
#cc = [np.corrcoef(np.vstack((ymort[ix],(ylos[ix]<th).astype(float))))[0,1] for th in Th]
#th = Th[argmax(cc)]
#ylosbin = (ylos<th).astype(float)
#llosbin = 'los<{0}'.format(th)

ysurgery = (np.mod(icutype,2)==0).astype(float)
assert(np.all(ysurgery==((icutype==2)|(icutype==4))))

ycardiac = (icutype<=2)
assert(np.all(ycardiac==(icutype<=2))).astype(int)

Y = np.vstack([ymort,ysurvbin,ysurgery,ycardiac]).T
Y_names = [ 'mortality', lsurvbin, 'surgery', 'cardiac' ]

print Y_names
print np.corrcoef(Y[ix,:].T)

In [ ]:
islabeled = np.all(~np.isnan(Y),axis=1) #& ~np.isnan(ylos)

In [ ]:
ym2 = Y[:,0].copy()
ym2[np.isnan(ym2)] = 2

fold10 = np.zeros((len(X),), dtype=int)
for fi,(_,teix) in enumerate(StratifiedKFold(ym2, n_folds=10)):
    fold10[teix] = fi+1

fold5 = np.zeros((len(X),), dtype=int)
for fi,(_,teix) in enumerate(StratifiedKFold(ym2, n_folds=5)):
    fold5[teix] = fi+1

In [ ]:
np.savez(os.path.join(args.output, 'physionet_challenge-60min.npz'),
            Xirreg=X, Tirreg=T,
            X=Ximpute, T=Timpute, Xmiss=Xmiss,
            Xu=XimputeUnit, Xlist=X_list,
            Xranges=ranges, Xstats=stats,
            V=Xstatic, Vmiss=Xstatic_miss, Vnames=Xstatic_names,
            Vranges=ranges_static,
            S=scores, Snames=scores_names,
            Y=Y, Ynames=Y_names,
            Ep=Ep, recordid=recordid, icutype=icutype,
            fold10=fold10, fold5=fold5,
            islabeled=islabeled
        )

In [ ]:
sio.savemat(os.path.join(args.output, 'physionet_challenge-60min.mat'),
          { 'Xirreg': X, 'Tirreg': T,
            'X': Ximpute, 'T': Timpute, 'Xmiss': Xmiss,
            'Xu': XimputeUnit, 'Xlist': X_list,
            'Xranges': ranges, 'Xstats': stats,
            'V': Xstatic, 'Vmiss': Xstatic_miss, 'Vnames': Xstatic_names,
            'Vranges': ranges_static,
            'S': scores, 'Snames': scores_names,
            'Y': Y, 'Ynames': Y_names,
            'Ep': Ep, 'recordid': recordid, 'icutype': icutype,
            'fold10': fold10, 'fold5': fold5,
            'islabeled': islabeled }
        )